In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import boto3
from botocore.config import Config

from DataFin import FMPClient, S3Client

import os
import dotenv
import boto3
import json
import pandas as pd
import io

dotenv.load_dotenv()

fmp_api_key = os.getenv('FMP_API_KEY')

s3_bucket = os.getenv('PERSONAL_S3_BUCKET_NAME')
aws_access_key =  os.getenv('PERSONAL_AWS_ACCESS_KEY_ID')
aws_secret_key =  os.getenv('PERSONAL_AWS_SECRET_ACCESS_KEY_ID')


fmp = FMPClient(fmp_api_key)

personal_s3 = S3Client(
    aws_account_access_key = aws_access_key,
    aws_secret_account_access_key = aws_secret_key,
    region = 'us-east-1',
    bucket_name = s3_bucket
)

polygon_aws_access_key = os.getenv('POLYGON_AWS_ACCESS_KEY_ID')
polygon_aws_secret_key = os.getenv('POLYGON_AWS_SECRET_ACCESS_KEY_ID')


session = boto3.Session(
   aws_access_key_id = polygon_aws_access_key,
   aws_secret_access_key = polygon_aws_secret_key
)

s3 = session.client(
   's3',
   endpoint_url='https://files.polygon.io',
   config=Config(signature_version='s3v4'),
)



from datetime import datetime, timedelta
now = datetime.now()
yesterday = now - timedelta(days=1)

year = now.year  
month = f'{now.month:02d}'
day = f'{now.day:02d}'
yesterday = f'{yesterday.day:02d}'
print(month, day, yesterday)


bucket_name = 'flatfiles'
object_key = f'us_stocks_sip/minute_aggs_v1/2025/{month}/2025-{month}-{yesterday}.csv.gz'

obj = s3.get_object(Bucket=bucket_name, Key=object_key)


df = pd.read_csv(io.BytesIO(obj['Body'].read()), compression='gzip')

04 23 22


In [4]:
personal_s3 = S3Client(
    aws_account_access_key = aws_access_key,
    aws_secret_account_access_key = aws_secret_key,
    region = 'us-east-1',
    bucket_name = s3_bucket
)

In [5]:
df['date_time'] = pd.to_datetime(df['window_start'], unit='ns').dt.tz_localize('UTC').dt.tz_convert('America/New_York').dt.tz_localize(None)

In [6]:
import pandas as pd
import boto3
import io
from pyarrow import parquet as pq
import pyarrow as pa



def write_parquet_to_s3(df, bucket, key):
    table = pa.Table.from_pandas(df)
    buffer = io.BytesIO()
    
    pq.write_table(table, buffer)
    buffer.seek(0)
    
    s3_client = boto3.client(
        's3',
        aws_access_key_id= aws_access_key,
        aws_secret_access_key= aws_secret_key,
        region_name='us-east-1'
    )
    
    s3_client.upload_fileobj(buffer, bucket, key)
    print(f"File uploaded to s3://{bucket}/{key}")



def read_parquet_with_boto3(bucket, key):

    s3_client = boto3.client(
        's3',
        aws_access_key_id= aws_access_key,
        aws_secret_access_key= aws_secret_key,
        region_name='us-east-1'
    )
    
    response = s3_client.get_object(
        Bucket=bucket,
        Key=key
    )
    
    buffer = io.BytesIO(response['Body'].read())
    
    table = pq.read_table(buffer)
    
    df = table.to_pandas()
    
    return df


write_parquet_to_s3(df, s3_bucket, 'dev/parquet-test/2025-04-21.parquet')


df_p = read_parquet_with_boto3(s3_bucket, 'dev/parquet-test/2025-04-21.parquet')

File uploaded to s3://data-fin-data-lake-test-1/dev/parquet-test/2025-04-21.parquet


In [7]:
df_p = read_parquet_with_boto3(s3_bucket, 'dev/parquet-test/2025-04-22.parquet')

In [8]:
df_p

,ticker,volume,open,close,high,low,window_start,transactions,date_time
0,CLSKW,1600,0.1910,0.1910,0.1965,0.1910,1745330100000000000,5,2025-04-22 09:55:00
1,CLSKW,460,0.1900,0.1900,0.1900,0.1900,1745330280000000000,4,2025-04-22 09:58:00
2,CLSKW,8000,0.1965,0.1980,0.1980,0.1965,1745330340000000000,6,2025-04-22 09:59:00
3,CLSKW,1400,0.1901,0.1901,0.1901,0.1901,1745330460000000000,2,2025-04-22 10:01:00
4,CLSKW,1400,0.1900,0.1900,0.1900,0.1900,1745330520000000000,3,2025-04-22 10:02:00
...,...,...,...,...,...,...,...,...,...
1660976,JMIA,300,2.1500,2.1400,2.1500,2.1400,1745324520000000000,2,2025-04-22 08:22:00
1660977,JMIA,201,2.1500,2.1500,2.1500,2.1500,1745324580000000000,1,2025-04-22 08:23:00
1660978,JMIA,2812,2.1500,2.1500,2.1500,2.1500,1745324640000000000,2,2025-04-22 08:24:00
1660979,JMIA,500,2.1487,2.1487,2.1487,2.1487,1745325840000000000,1,2025-04-22 08:44:00
